## Scenario 1: A single data scientist participating in an ML competition

MLflow setup:
* Tracking server: no
* Backend store: local filesystem
* Artifacts store: local filesystem

The experiments can be explored locally by launching the MLflow UI.

In [2]:
import mlflow

In [3]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///home/jovyan/work/mlruns'


In [4]:
# mlflow.list_experiments()   # Deprecated
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1727433801456, experiment_id='0', last_update_time=1727433801456, lifecycle_stage='active', name='Default', tags={}>]

### Creating an experiment and logging a new run

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import os 
mlflow.set_experiment("my-experiment-1")

with mlflow.start_run() as run:

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    
    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/09/27 10:43:31 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2024/09/27 10:43:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'file:///home/jovyan/work/mlruns/974795307949675102/ba1c744c7ce64295af7dc85b4dd25643/artifacts'


In [6]:
# mlflow.list_experiments()   # Deprecated
mlflow.search_experiments()

[<Experiment: artifact_location='file:///home/jovyan/work/mlruns/974795307949675102', creation_time=1727433811912, experiment_id='974795307949675102', last_update_time=1727433811912, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1727433801456, experiment_id='0', last_update_time=1727433801456, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [7]:
from mlflow.tracking import MlflowClient


client = MlflowClient()

In [28]:

# Storing models in mlflow using local file storage was 
# not possible on the earlier versions of mlflow. Now it is.
"""
from mlflow.exceptions import MlflowException

try:
    client.list_registered_models()
except MlflowException:
    print("It's not possible to access the model registry :(")
"""
# run_id = client.list_run_infos(experiment_id='1')[0].run_id

experiment_id = client.search_experiments(filter_string="name = 'my-experiment-1'")[0].experiment_id
run_id = client.search_runs(experiment_ids=experiment_id)[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

client.search_registered_models()

Registered model 'iris-classifier' already exists. Creating a new version of this model...
Created version '1' of model 'iris-classifier'.


[<RegisteredModel: aliases={}, creation_timestamp=1727434425788, description=None, last_updated_timestamp=1727434475684, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1727434475684, current_stage='None', description=None, last_updated_timestamp=1727434475684, name='iris-classifier', run_id='ba1c744c7ce64295af7dc85b4dd25643', run_link=None, source='file:///home/jovyan/work/mlruns/974795307949675102/ba1c744c7ce64295af7dc85b4dd25643/artifacts/models', status='READY', status_message=None, tags={}, user_id=None, version=1>], name='iris-classifier', tags={}>]